In [1]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar

#Logger
logging.basicConfig(filename='Extraction.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [2]:
##############
# PARAMETERS #
##############

client = MongoClient(username='XXX', password='XXX')

DatabaseName = "StreamingHashtags"
CollectionName = "Campanya-Interactions"

In [3]:
##############################
# BUILDING GRAPH - VERSION 2 #
##############################

COMMUNITIES_LIST = ["VOX", "JXCAT", "CUP", "COMUNS", "PSC", "ERC", "PP", "CS"]
#COMMUNITIES_LIST = ["VOX", "POR_AND", "PSOE", "PP", "ADELANTE_AND", "CS"]
#COMMUNITIES_LIST = ["VOX", "PODEMOS", "PSOE", "PP", "MAS_MAD", "CS"]

community_dict = {}

G = nx.DiGraph()

db = client[DatabaseName]

tweets = db[CollectionName].find(no_cursor_timeout=True, batch_size=1000000)

for result in tweets:
    active_community = result['ACTIVE_community']
    passive_community = result['PASSIVE_community']
    user_name = result['user']['screen_name']
    
    if active_community in COMMUNITIES_LIST:
        if passive_community in COMMUNITIES_LIST:
            if 'retweeted_status' in result:
                G.add_node(user_name)

                if G.has_edge(user_name, result['retweeted_status']['user']['screen_name']):
                    G[user_name][result['retweeted_status']['user']['screen_name']]['weight'] += 1.0
                else:
                    G.add_edge(user_name, result['retweeted_status']['user']['screen_name'], weight = 1.0) 
                    
                if user_name not in community_dict:
                    community_dict[user_name] = active_community
                    
                if result['retweeted_status']['user']['screen_name'] not in community_dict:
                    community_dict[result['retweeted_status']['user']['screen_name']] = passive_community

tweets.close()                    
                    
print("Nombre de nodes: {}".format(G.number_of_nodes()))
print("Nombre d'arestes: {}".format(G.number_of_edges()))

Nombre de nodes: 138518
Nombre d'arestes: 843546


In [4]:
## Change Cat-Mad-And depending on the case under study

nx.write_gml(G, "Cat-CI-Complete.gml")

len(community_dict)

with open('Cat-community_dict.json', 'w') as fp:
    json.dump(community_dict, fp)